<a href="https://colab.research.google.com/github/KaizenTherapy/large-language-models/blob/main/LoRa_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install -q bitsandbytes datasets accelerate loralib
!pip install transformers peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 3.6 MB/s eta 0:00:00


In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!nvidia-smi -L

In [6]:
import os
os.environ["CUDA_VISIBLE_DEVICE"]="0"

import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    "bigscience/bloom-560m",
    load_in_8bit = True,
    device_map = "auto",
)

tokenizer = AutoTokenizer.from_pretrained("bigscience/bloom-560m")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

RuntimeError: No GPU found. A GPU is needed for quantization.

In [ ]:
# freeze weights

for param in model.parameters():
  param.requires_grad = False # freeze the model - will train adapters

  if param.ndim = 1:
    # cast small param like layer notm to fp32 for stability
    param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable() # reduce number of stored activations
model.enable_input_require_grads()

class CastOutputToFloat(nn.Sequential):

  def forward(self, x):
    return super().forward(x).to(torch.float32)

  model.lm_head = CastOutputToFloat(model.lm_head)


In [ ]:
# set up LoRa Adapators

def print_trainable_parameters(model):
  """
  prints number of trainable parameters in the model
  """

  trainable_params = 0
  all_param = 0

  for _, param in model.named_parameters():
    all_param += param.numel()

    if param.requires_grad:
      trainable_param += param.numel()

    print(
        f"""
        trainable params: {trainable_params} || all params: {all_param} || trainable% {100 * trainable_params / all_param}
         """
    )

In [ ]:
from peft import LoraConfig, get_peft_model


config = LoraConfig(
    r= 16, # attention heads
    lora_alpha =32, # alpha sclaing
    lora_dropout = 0.05,
    bias = "none"
    task_type = "CAUSAL_LM" #set this for CLM or Seq2Seq
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

In [ ]:
import transformers
from datasets import load_dataset

data = load_dataset("Abirate/english_quotes")

In [ ]:
def merge_columns(example):
  example["prediction"] = example["quote"] +"->:" + str(example["tags"])

  data["train"] = data["train"].map(merge_columns)
  data["train"]["prediction"][:5]


In [ ]:
data["train"][0]

In [ ]:
data = data.map(lambda samples: tokenizer(samples['prediction']), batched = True)

In [ ]:
data

In [ ]:
# Training

trainer = transformers.Trainer(
    model= model,
    train_dataset = data['train'],
    args = transformers.TrainingArguments(
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 4,
        warmup_steps = 100,
        max_steps = 200,
        learning_rate = 2e-4,
        fp16 = True,
        logging_steps = 1,
        output_dir = "outputs"
    ),

    data_collator = transformers.DataCollatorForLanguageModeling(tokenizer, mlm = False)

)

model.config.use_cache = False

trainer.train()

In [ ]:
model.push_to_hub("kaizen555/bloom-560m",
                  use_auth_token = True,
                  commit_message = "basic training",
                  private = True)

In [ ]:
# load adapter

import torch
import peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

peft_model_id = "kaizen555/bloom-560m"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, return_dict = True, load_in_8bit= True, device_map = "auto" )
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# load Lora model
model = PeftModel.from_pretained(model, peft_mode_id)

In [1]:
batch = tokenizer(""training model with PEFT/LORA is cool" ->: ", return_tensors= "pt")

with torch.cuda.amp.autocast():
  output_tokens = model.generate(output_tokens[0], skip_special_tokens = True)

SyntaxError: invalid syntax. Perhaps you forgot a comma? (<ipython-input-1-5a993d1ae2ae>, line 1)